In [1]:
import numpy as np
import equistore
from equistore.operations import mean_over_samples, slice
from rascaline import SphericalExpansion, SoapPowerSpectrum
#import chemiscope
import ase.io as aseio
from scipy import optimize

import copy as copy
from ase.build.tools import sort as asesort

from rascal.models import Kernel, train_gap_model, compute_KNM
from rascal.representations import SphericalInvariants
from rascal.representations import SphericalExpansion as SEXP
from rascal.utils import from_dict, to_dict, CURFilter, dump_obj, load_obj
from rascal.neighbourlist.structure_manager import (
        mask_center_atoms_by_species, mask_center_atoms_by_id)

import sys
import argparse
import json




In [2]:
def mk_diff(A,B):
    blocks = []
    for key,block in A:
        values = block.values - B[key].values
        blocks.append(equistore.TensorBlock(values=values,
                                            samples=B[key].samples,
                                           properties=block.properties,
                                           components=block.components))
    return equistore.TensorMap(A.keys,blocks)

def mk_dot(A,B):
    A1 = A.components_to_properties('spherical_harmonics_m').keys_to_properties('spherical_harmonics_l')
    B1 = B.components_to_properties('spherical_harmonics_m').keys_to_properties('spherical_harmonics_l')
    return np.dot(A1[0].values.reshape(1,-1),B1[0].values.reshape(-1)).reshape(1)[0]
def mk_dot_perL(A,B):
    A1 = A.components_to_properties('spherical_harmonics_m')
    B1 = B.components_to_properties('spherical_harmonics_m')
    res = 0
    for i in range(len(B1.keys)):
        res += np.sum((A1.block(i).values* B1.block(i).values))**2
    return res



In [3]:

hypers = {
    "cutoff": 7,
    "max_radial": 9,
    "max_angular": 5,
    "atomic_gaussian_width": 1.0,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0
}

alpha = 0.3

model = "./models/model-PBEsol-menocoesiveenergy-rcut5_sigma0.3-4000sparseenv-1500traindata.json" 
inputfile = "./data/beta-gamma_aligned_sorted.extxyz"

delta = 0.1


In [4]:

# use "aligned" structures
align_beta, rot_gamma = aseio.read(inputfile,":")#aseio.read("/tmp/beta-gamma.extxyz",":")
beta, gamma = align_beta, rot_gamma

print('letto tutto')
print('alpha',alpha)

calculator = SphericalExpansion(**hypers)
#calculator = SoapPowerSpectrum(**hypers)



letto tutto
alpha 0.3


In [5]:
def mk_descriptor(frame, calculator=calculator):
    descriptor = calculator.compute([frame])
    descriptor = descriptor.keys_to_properties(['species_neighbor',]).keys_to_samples('species_center')
    descriptor = slice(descriptor, samples=equistore.Labels(["species_center"], np.array([[15],[16]], dtype=np.int32)),
        properties=equistore.Labels(["species_neighbor"], np.array([[15],[16]], dtype=np.int32)))
    return mean_over_samples(descriptor, samples_names = ['center'])


def mk_frame(pos, abcdef, template=beta):
    frame = template.copy()
    frame.positions = pos.reshape(-1,3)
    # keep the cell orthorhombic
    frame.cell = abcdef[:3]
    # frame.cell = [[abcdef[0],0,0],[abcdef[3],abcdef[1],0],[abcdef[4],abcdef[5],abcdef[2]]]
    return frame


In [6]:
descriptor = calculator.compute([beta, gamma, align_beta, rot_gamma])


descriptor = descriptor.keys_to_properties(['species_neighbor']).keys_to_samples('species_center')
descriptor = slice(descriptor, samples=equistore.Labels(["species_center"], np.array([[15],[16]], dtype=np.int32)),
        properties=equistore.Labels(["species_neighbor"], np.array([[15],[16]], dtype=np.int32)))

mean_descriptor = mean_over_samples(descriptor, samples_names = ['center'])


desc_beta = slice(mean_descriptor, samples=equistore.Labels(["structure"], np.array([[0]], np.int32)))
desc_gamma = slice(mean_descriptor, samples=equistore.Labels(["structure"], np.array([[1]], np.int32)))
print('desc_beta',desc_beta)
print('desc_gamma',desc_gamma)

#diff_gammabeta = mk_diff(desc_gamma,desc_beta)
diff_betagamma = mk_diff(desc_beta,desc_gamma)
print('diff_gammabeta',diff_betagamma)
betagamma2 = mk_dot_perL(diff_betagamma,diff_betagamma)



desc_beta TensorMap with 6 blocks
keys: ['spherical_harmonics_l']
                  0
                  1
                  2
                  3
                  4
                  5
desc_gamma TensorMap with 6 blocks
keys: ['spherical_harmonics_l']
                  0
                  1
                  2
                  3
                  4
                  5
diff_gammabeta TensorMap with 6 blocks
keys: ['spherical_harmonics_l']
                  0
                  1
                  2
                  3
                  4
                  5


In [7]:


PBEsolpot=load_obj(model)
energiesPBEsol = []
forcesPBEsol = []

soapPBEsol = PBEsolpot.get_representation_calculator()


In [8]:

n_eval = 0
intermediates = []

def collective_variable_milestone(pos_cell, target_descriptors,init_m_target,cv_milestone,alpha_energy=1,ref_energy=-7.863632805093914):
    global n_eval
    pos = pos_cell[:-6]
    cell = pos_cell[-6:]  
    frame = mk_frame(pos, cell)
    desc = mk_descriptor(frame)
    eta_m_target = mk_diff(target_descriptors,desc)
    s = mk_dot_perL(eta_m_target,init_m_target)
    frame.wrap(eps=1e-10)
    m = soapPBEsol.transform(frame)
    energy=0
    energy = PBEsolpot.predict(m)[0]
    diff = (s-cv_milestone)**2
    energy_contrib = alpha_energy*(energy-ref_energy)
    diff += energy_contrib
    if n_eval%100 ==0:
        print(n_eval,cell, diff,energy)
        if n_eval%100 == 0:
            frame.info["totdiff"] = diff
            frame.info["diff"] = diff-energy_contrib
            frame.info["energy"] = energy
            frame.positions -= frame.positions[12]
            frame.wrap(eps=1e-8)
            intermediates.append(frame)
    n_eval += 1 
    return diff


In [ ]:


print(f'{1./betagamma2=}')
diff_betagamma_normalized = equistore.operations.multiply(diff_betagamma,1./betagamma2)
x0 = np.concatenate([gamma.positions.flatten(), gamma.cell.diagonal(), [0,0,0]])
start = 0
print(np.linspace(0,1,int(1/delta)+1)[1:])
for factor in np.linspace(0,1,int(1/delta)+1)[1:]:
    start=len(intermediates)-start
    #tmp  = equistore.operations.multiply(diff_gammabeta , factor
    desc_milestone = equistore.operations.add(equistore.operations.multiply(diff_betagamma , factor),desc_gamma)
    beta_m_milestone = mk_diff(desc_beta,desc_milestone)
    s_milestone = mk_dot_perL(beta_m_milestone,diff_betagamma_normalized)
    for i in range(8):
        find_struc = optimize.minimize(collective_variable_milestone, args=(desc_beta,diff_betagamma_normalized,s_milestone,alpha,-7.8636328052103295), x0=x0, method="Nelder-Mead", options={"maxfev":15000, "initial_simplex":x0+0.01*np.random.normal(size=(len(x0)+1,len(x0)))})
        x0 = np.concatenate([intermediates[-1].positions.flatten(), intermediates[-1].cell.diagonal(), [0,0,0]])
        aseio.write( f'./traj_minimizzation-script_aligned_sorted_long_a{alpha}_target{factor}.extxyz', intermediates[start:])
        if (intermediates[-1].info["diff"]<1e-6):
            break

aseio.write(f'./traj_minimizzation-script_aligned_sorted_long_a{alpha}_total.extxyz', intermediates)


In [ ]:
cs = chemiscope.show(frames=[align_beta]+intermediates+[rot_gamma], 
                     properties=dict(
                      index = list(range(len(intermediates)+2)),
                      volume = [beta.cell.volume/len(beta)]+[i.cell.volume/len(i) for i in intermediates] + [gamma.cell.volume/len(gamma)],
                      diff = [beta_diff_librascal]+[i for i in diff] + [0]
                     ),
                    settings={'map': {'x': {'max': 12.82862019789748,
   'min': -0.8286201978974813,
   'property': 'index',
   'scale': 'linear'},
  'y': {'max': 27.476935995554605,
   'min': -2.1041542922071805,
   'property': 'diff',
   'scale': 'log'},
  'z': {'max': 26.165681131577035,
   'min': -0.792899428229607,
   'property': '',
   'scale': 'linear'},
  'color': {'max': 20.81712071720822,
   'min': 18.79932584713234,
   'property': 'volume',
   'scale': 'linear'},
  'symbol': '',
  'palette': 'inferno',
  'size': {'factor': 50, 'mode': 'linear', 'property': '', 'reverse': False}},
'structure': [{'bonds': True,
   'spaceFilling': False,
   'atomLabels': False,
   'unitCell': True,
   'rotation': False,
   'supercell': {'0': 3, '1': 2, '2': 2},
   'axes': 'abc',
   'keepOrientation': True,
   'playbackDelay': 100,
   'environments': {'activated': True,
    'bgColor': 'grey',
    'bgStyle': 'ball-stick',
    'center': False,
    'cutoff': 0}}]}
                    )
display(cs)